In [29]:
from typing  import Generator
from helpers import data

In [7]:
start = data(17)
start

['#.#####.',
 '#..##...',
 '.##..#..',
 '#.##.###',
 '.#.#.#..',
 '#.##..#.',
 '#####..#',
 '..#.#.##']

In [4]:
len(start), len(start[0])

(8, 8)

A **neighbor** is any cube where **any coordinate differs by at most 1**. 

Active cubes are denoted with \#, and inactive cubes are denoted with \. 

The boot process consists of six cycles. During each cycle: 
- If a cube is **active** and exactly **2 or 3 neighbors are active**, it stays active. Otherwise it becomes inactive. 
- If a cube is **inactive** and exactly **3 neighors are active**, it becomes active. Otherwise it stays inactive. 

**Part 1:** How many cubes are left in the active state after the sixth cycle?

In [82]:
class MutableBool:
    def __init__(self, val=False):
        self.val = val 
        self.next = False
        
    def __bool__(self):
        return self.val
    
    def __repr__(self):
        return str(self.val)
    
    def update(self):
        self.val, self.next = self.next, False

In [83]:
# At each time step, update each entry based on the rule and its neighbors 
class NGrid(dict):
    """NGrid will choose default values for missing keys and is nested n deep."""
    def __init__(self, n):
        self.n = n
        
    def __missing__(self, key): 
        if self.n == 1:
            self[key] = MutableBool()
        else: 
            self[key] = NGrid(self.n - 1)
        return self[key]
    
    def life(self, steps): 
        """Simulate steps iterations of Conway's Game of Life."""
        for i in range(steps): 
            self.step()
    
    def step(self):
        pass
        
    def neighbors(self, *args) -> Generator[bool, None, None]:
        """Yield neighbors of coordinate at self[args[0]][args[1]][...]
        
        Actually, I wasn't able to do this without yielding self too...
        """
        if len(args) != self.n: 
            raise ValueError(f"Expected {self.n} integer coordinates, got {len(args)}.")
        if any(type(arg) != int for arg in args):
            print(args)
            
            print([type(arg) for arg in args])
            raise ValueError("Expected integer coordinates.")
            
        first, *rest = args
        if self.n == 1:
            yield self[first - 1]
            yield self[first]
            yield self[first + 1]
        else: 
            yield from self[first - 1].neighbors(*rest)
            yield from self[first].neighbors(*rest)
            yield from self[first + 1].neighbors(*rest)

In [84]:
g = NGrid(1)

In [85]:
list(g.neighbors(1))

[False, False, False]

In [42]:
new = [
    ['.', '.', '.'],
    ['.', '.', '.'],
    ['.', '.', '.'],
]
for i in range(3):
    for j in range(3):
        active_neighbors = sum(n == '#' for n in neighbors(i, j, 0, example))
        if example[i][j] == '#': 
            if 2 <= active_neighbors <= 3:
                new[i][j] = '#'
        else:
            if active_neighbors == 3:
                new[i][j] = '#'